In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/wids2023manar/train_data.csv
/kaggle/input/wids2023manar/test_data.csv
/kaggle/input/wids2023manar/sample_solution.csv


In [ ]:
pip -q install xgboost

In [7]:
pip install catboost

Note: you may need to restart the kernel to use updated packages.


In [5]:
import xgboost

In [4]:
import catboost

In [66]:
import pandas as pd

In [ ]:
# for i in train.columns:
#     print(i," ", train[i].isnull().sum())

In [67]:
train=pd.read_csv('/kaggle/input/wids2023manar/train_data.csv')
test=pd.read_csv('/kaggle/input/wids2023manar/test_data.csv')

In [92]:
train.dropna(inplace=True)
test.dropna(inplace=True)

In [93]:
train.climateregions__climateregion.replace(\
            {'BSh':1, 'BSk':1, 'BWk':1, 'BWh':1, \
              'Cfa':2, 'Csa':2, 'Csb':2, 'Cfb':2, 'Dfb':3\
, 'Dsc':3, 'Dfc':3, 'Dfa':3, 'Dsb':3, 'Dwa':3, 'Dwb':3}, inplace=True)
test.climateregions__climateregion.replace(\
            {'BSh':1, 'BSk':1, 'BWk':1, 'BWh':1, \
              'Cfa':2, 'Csa':2, 'Csb':2, 'Cfb':2, 'Dfb':3\
, 'Dsc':3, 'Dfc':3, 'Dfa':3, 'Dsb':3, 'Dwa':3, 'Dwb':3}, inplace=True)

from datetime import datetime
train['startdate'] = pd.to_datetime(train.startdate, infer_datetime_format=True)
test['startdate'] = pd.to_datetime(test.startdate, infer_datetime_format=True)

train['year'] = train['startdate'].dt.year
train['month'] = train['startdate'].dt.month
train['day'] = train['startdate'].dt.day
test['year'] = test['startdate'].dt.year
test['month'] = test['startdate'].dt.month
test['day'] = test['startdate'].dt.day

In [97]:
train_y=train['contest-tmp2m-14d__tmp2m']
train_X=train.drop(columns=['contest-tmp2m-14d__tmp2m', 'index'], axis=1)
test_X=test.drop(columns=['index'])

In [98]:
integers=train_X.select_dtypes(include='int64').columns

In [99]:
floats=train_X.select_dtypes(include='float64').columns

In [133]:
numerics=floats.append(integers)
wanted=numerics.drop('year', 'day')

In [34]:
objects=train_X.select_dtypes(include='O').columns
# train=pd.get_dummies(train, columns=['climateregions__climateregion'])
# test=pd.get_dummies(test, columns=['climateregions__climateregion'])

In [ ]:
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [113]:
from catboost import CatBoostRegressor, Pool

In [78]:
from sklearn.linear_model import LinearRegression

In [108]:
model=CatBoostRegressor(n_estimators= 100)

In [134]:
model.fit(Pool(train_X[wanted], train_y))

Learning rate set to 0.5
0:	learn: 5.7803334	total: 336ms	remaining: 33.3s
1:	learn: 3.8177979	total: 632ms	remaining: 31s
2:	learn: 2.8767401	total: 969ms	remaining: 31.3s
3:	learn: 2.4511923	total: 1.27s	remaining: 30.4s
4:	learn: 2.2487835	total: 1.55s	remaining: 29.5s
5:	learn: 2.1053364	total: 1.86s	remaining: 29.2s
6:	learn: 1.9976085	total: 2.21s	remaining: 29.4s
7:	learn: 1.9307208	total: 2.55s	remaining: 29.4s
8:	learn: 1.8578832	total: 2.89s	remaining: 29.2s
9:	learn: 1.8190877	total: 3.22s	remaining: 29s
10:	learn: 1.7663058	total: 3.65s	remaining: 29.5s
11:	learn: 1.7299285	total: 3.89s	remaining: 28.5s
12:	learn: 1.6965008	total: 4.12s	remaining: 27.6s
13:	learn: 1.6517122	total: 4.5s	remaining: 27.7s
14:	learn: 1.6126470	total: 4.95s	remaining: 28.1s
15:	learn: 1.5835711	total: 5.42s	remaining: 28.5s
16:	learn: 1.5474007	total: 5.73s	remaining: 28s
17:	learn: 1.5181495	total: 5.99s	remaining: 27.3s
18:	learn: 1.4997581	total: 6.17s	remaining: 26.3s
19:	learn: 1.4858283	to

In [135]:
df=pd.DataFrame(model.get_feature_importance(Pool(train_X[wanted])))
df['features']=train_X[wanted].columns
df

,0,features
0,2.140522,lat
1,3.061672,lon
2,0.825900,contest-pevpr-sfc-gauss-14d__pevpr
3,0.000000,nmme0-tmp2m-34w__cancm30
4,0.058192,nmme0-tmp2m-34w__cancm40
...,...,...
240,0.000000,mjo1d__phase
241,0.000000,mei__meirank
242,0.000000,mei__nip
243,0.560649,month


In [136]:
predicted=model.predict(test_X[wanted])

In [102]:
dataframe=pd.DataFrame({'contest-tmp2m-14d__tmp2m':predicted, 'index':test['index']})

In [103]:
dataframe.to_csv('sub1.csv', index=False)